useful links:

https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/get-token-count

#### **simple response**

In [6]:
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

def generate_function_call(prompt: str) -> str:
    # Load the Vertex AI Gemini API to use function calling
    model = GenerativeModel("gemini-pro")

    # Specify a function declaration and parameters for an API request
    get_current_weather_func = FunctionDeclaration(
        name="get_current_weather",
        description="Get the current weather in a given location",
        # Function parameters are specified in OpenAPI JSON schema format
        parameters={
            "type": "object",
            "properties": {"location": {"type": "string", "description": "Location"}},
        },
    )

    # Define a tool that includes the above get_current_weather_func
    weather_tool = Tool(
        function_declarations=[get_current_weather_func],
    )

    # Prompt to ask the model about weather, which will invoke the Tool
    prompt = prompt

    # Instruct the model to generate content using the Tool that you just created:
    response = model.generate_content(
        prompt,
        generation_config={"temperature": 0},
        tools=[weather_tool],
    )

    # Transform the structured data into a Python dictionary
    params = {}
    for key, value in response.candidates[0].content.parts[0].function_call.args.items():
        params[key] = value
    params

    # This is where you would make an API request to get the location of the store closest to the user.
    # Here we'll use synthetic data to simulate a response payload from an external API.
    api_response = """{ "location": "Boston, MA", "temperature": 38, "description": "Partly Cloudy",
                  "icon": "partly-cloudy", "humidity": 65, "wind": { "speed": 10, "direction": "NW" } }"""

    # Return the API response to Gemini so it can generate a model response or request another function call
    response = model.generate_content(
        [
        Content(role="user", parts=[
            Part.from_text(prompt),
        ]),
        Content(role="function", parts=[
            Part.from_dict({
                "function_call": {
                    "name": "get_current_weather",
                }
            })
        ]),
        Content(role="function", parts=[
            Part.from_function_response(
                name="get_current_weather",
                response={
                    "content": api_response,
                }
            )
        ]),
        ],
        tools=[weather_tool],
    )

    answer = response.candidates[0].content.parts[0].text

    return (response, params, answer)

#### **multi-turn**

In [7]:
from vertexai import generative_models
from vertexai.generative_models import GenerativeModel
model = GenerativeModel("gemini-pro")

get_current_weather_func = generative_models.FunctionDeclaration(
  name="get_current_weather",
  description="Get the current weather in a given location",
  parameters={
      "type": "object",
      "properties": {
          "location": {
              "type": "string",
              "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
              "type": "string",
              "enum": [
                  "celsius",
                  "fahrenheit",
              ]
          }
      },
      "required": [
          "location"
      ]
  },
)

weather_tool = generative_models.Tool(
  function_declarations=[get_current_weather_func],
)

function_calling_chat = model.start_chat()
model_response = function_calling_chat.send_message("What is the weather like?", tools=[weather_tool])
print("model_response\n", model_response)

model_response
 candidates {
  content {
    role: "model"
    parts {
      text: "For which location would you like to know the weather?"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 39
  candidates_token_count: 11
  total_token_count: 50
}



In [8]:
model_response = function_calling_chat.send_message("in boston please?", tools=[weather_tool])
print("model_response\n", model_response)

model_response
 candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_current_weather"
        args {
          fields {
            key: "location"
            value {
              string_value: "boston, ma"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 54
  candidates_token_count: 9
  total_token_count: 63
}



In [9]:
model_response = function_calling_chat.send_message(
  Part.from_function_response(
      name="get_current_weather",
      response={
          "content": {"weather_there": "super nice"},
      }
  ),
  tools=[weather_tool]
)
print("model_response\n", model_response)

model_response
 candidates {
  content {
    role: "model"
    parts {
      text: "The weather in boston, ma is super nice."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 74
  candidates_token_count: 10
  total_token_count: 84
}



In [5]:
function_calling_chat.history

[role: "user"
 parts {
   text: "What is the weather like?"
 },
 role: "model"
 parts {
   text: "For which location would you like to know the weather?"
 },
 role: "user"
 parts {
   text: "in boston please?"
 },
 role: "model"
 parts {
   function_call {
     name: "get_current_weather"
     args {
       fields {
         key: "location"
         value {
           string_value: "boston, ma"
         }
       }
     }
   }
 },
 role: "user"
 parts {
   function_response {
     name: "get_current_weather"
     response {
       fields {
         key: "content"
         value {
           struct_value {
             fields {
               key: "weather_there"
               value {
                 string_value: "super nice"
               }
             }
           }
         }
       }
     }
   }
 },
 role: "model"
 parts {
   text: "The weather there is super nice."
 }]